Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from random import choices
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from itertools import product
import pickle
import math
import numpy as np
from enum import Enum
import lab3_lib
from lab3_lib import AbstractProblem
from itertools import chain
import concurrent.futures


In [ ]:
class Agent():

    class Crossover(Enum):
        RANDOM = 1
        MULTI_CUT = 2
        SCRAMBLE = 3
        CYCLE = 4

    class AgentType(Enum):
        BINARY = 1
        PATTERN = 2

    def __init__(self,genome,mutation_rate : float = None) -> None:
        self.genome = genome
        self.mutation_rate = mutation_rate
        self._id = random.randint(0, 1000000000)
        self._fitness = None
        pass

    def set_mutation_rate(self, mutation_rate : float) -> None:
        self.mutation_rate = mutation_rate

    def set_genome(self, genome) -> None:
        self.genome = genome

    def compute_fitness(self,fitness_function) -> None:
        if self.genome is None:
            ValueError("Genome is not set")
        self._fitness = fitness_function(self.genome)
        pass  

    def mutation(self,mutation_rate) -> None:
        if self.genome is None:
            ValueError("Genome is not set")
        if mutation_rate is None and self.mutation_rate is None:
            ValueError("Mutation rate is not set")
        if self.mutation_rate is not None and mutation_rate is None:
            self.mutation_rate *= [0.999,1.001][random.randint(0,1)]
        pass      

    @property
    def fitness(self):
        if self._fitness is None:
            self.compute_fitness()
        return self._fitness

    def load_agent(self, path):
        with open(path, 'rb') as f:
            self.genome = pickle.load(f)
        return self
    
    def save_agent(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self.genome, f)

In [ ]:
class AgentGA(Agent):

    def __init__(self) -> None:
        super().__init__(None,None)

    def set_genome(self,genome_size : int,genome : list = None) -> Agent:
        if genome is None:
            genome = [random.uniform(0, 1) < 0.5 for _ in range(genome_size)]
        super().set_genome(genome)
        
        return self

    def set_mutation_rate(self,mutation_rate : float) -> Agent:
        super().set_mutation_rate(mutation_rate)

        return self

    def mutation(self,mutation_rate : float = None) -> Agent:
        super().mutation(mutation_rate)

        mask = [random.uniform(0, 1) < (mutation_rate if mutation_rate is not None else self.mutation_rate) for _ in range(len(self.genome))]
        self.genome = [a ^ b for a, b in zip(self.genome, mask)]
        return self

    def __repr__(self) -> str:
        return f"AgentGA({self._id},{self.fitness})"

    def crossover(self,other : Agent,crossover_type : Agent.Crossover = Agent.Crossover.RANDOM,other_weight = 0.5) -> Agent:
        match crossover_type:
            case Agent.Crossover.RANDOM:
                return self.random_crossover(other,other_weight)
            case Agent.Crossover.MULTI_CUT:
                return self.multi_cut_crossover(other)
            case Agent.Crossover.SCRAMBLE:
                return self.scramble_crossover(other)
            case Agent.Crossover.CYCLE:
                return self.cycle_crossover(other)
            case _:
                raise ValueError("Unknown crossover type")

    def random_crossover(self,other : Agent,other_weight = 0.5) -> Agent:
        #print("crossover")
        for (i,_) in enumerate(self.genome):
            self.genome[i] = random.choices([self.genome[i],other.genome[i]],weights=[1-other_weight,other_weight])[0]
        return self
    
    def multi_cut_crossover(self,other : Agent) -> Agent:
        #print("multi_cut_crossover")
        n = random.randint(1, len(self.genome)//2-1)
        size = len(self.genome)//n
        for i in range(n):
            start = i * size
            end = (i + 1) * size - 1
            n = random.randint(start, end)
            m = random.randint(start, end)
            if n > m:
                n,m = m,n
            self.genome[n:m] = other.genome[n:m]
        return self

    def scramble_crossover(self,other : Agent) -> Agent: #TODO rewrite and rename as One Point Crossover
        #print("scramble_mutation")
        n = random.randint(0, len(self.genome)-1)
        self.genome[n:] = other.genome[n:]
        return self
    
    def cycle_crossover(self,other : Agent) -> Agent:
        #print("cycle_crossover")
        n = random.randint(0, len(self.genome)-1)
        m = random.randint(0, len(self.genome)-1)
        if n > m:
            n,m = m,n
        self.genome[n:m] = other.genome[n:m]
        return self

    def compute_fitness(self,fitness_function : AbstractProblem) -> None:
        super().compute_fitness(fitness_function)
        #print(f"compute_fitness : {fitness_function.calls}")
        

In [ ]:
class PatternBasedAgent(Agent):

    def __init__(self) -> None:
        #self.pattern_size = random.randint(1,genome_size//10)
        super().__init__(None,None)
        
    def set_genome(self,genome_size : int,genome : list = None) -> Agent:
        self.genome_size = genome_size
        self.pattern_size = random.randint(genome_size//50,genome_size//5)
        self.pattern = [random.uniform(0, 1) < 0.5 for _ in range(self.pattern_size)]
        self.generate_genome()
        return self
    
    def set_mutation_rate(self,mutation_rate : float) -> Agent:
        super().set_mutation_rate(mutation_rate)
        self._pattern_mutation_rate = min(mutation_rate * self.pattern_size / 10 , 0.2)
        return self

    def generate_genome(self):
        self.genome = list(chain.from_iterable([self.pattern for _ in range(self.genome_size//self.pattern_size + 1)])) # overshoot the size to be sure to have enough
        self.genome = self.genome[:self.genome_size]
        #print(f"size : {self.pattern_size} with pattern : {self.pattern} , genome : {self.genome}")

    def mutation(self,mutation_rate : float = None) -> Agent:
        super().mutation(mutation_rate)

        if random.random() < (self._pattern_mutation_rate if mutation_rate is None else mutation_rate * 10) :
            adding = random.choices([-1, 1], weights=[1, 2])[0]
            if adding == -1 and self.pattern_size > 1:
                self.pattern_size -= 1
                self.pattern = self.pattern[:-1]
            else:
                
                temp = random.choices([0, 1], k = adding)
                self.pattern_size += adding
                self.pattern += temp
        else:
            for (i,state) in enumerate(self.pattern):
                if random.random() < (mutation_rate if mutation_rate is not None else self.mutation_rate) :
                    self.pattern[i] = not state
        self.generate_genome()
        return self

    def __repr__(self) -> str:
        return f"AgentGA({self._id},{self.fitness}) with pattern {self.pattern}"

    def crossover(self,other : Agent,crossover_type : Agent.Crossover = 1,other_weight = 0.5) -> Agent:
        match crossover_type:
            case Agent.Crossover.RANDOM:
                return self.random_crossover(other,other_weight)
            case Agent.Crossover.MULTI_CUT:
                return self.multi_cut_crossover(other)
            case Agent.Crossover.SCRAMBLE:
                return self.one_cut_crossover(other)
            case Agent.Crossover.CYCLE:
                return self.cycle_crossover(other)
            case _:
                raise ValueError("Unknown crossover type")

    def random_crossover(self,other : Agent,other_weight = 0.5) -> Agent:
        #print("crossover")
        offset = self.pattern_size - other.pattern_size
        if offset > 0:
            temp = random.randint(0,offset)
            for (i,_) in enumerate(other.pattern):
                self.pattern[i + temp] = random.choices([self.pattern[i + temp],other.pattern[i]],weights=[1-other_weight,other_weight])[0]
        else:
            temp = random.randint(0,abs(offset))
            for (i,_) in enumerate(self.pattern):
                self.pattern[i] = random.choices([self.genome[i],other.genome[i + temp]],weights=[1-other_weight,other_weight])[0]
        self.generate_genome()
        return self
    
    def multi_cut_crossover(self,other : Agent) -> Agent:
        #print("multi_cut_crossover")
        for _ in range(random.randint(0, (min(self.pattern_size , other.pattern_size) - 1)//2)):
            n = random.randint(0, min(self.pattern_size , other.pattern_size) - 1)
            m = random.randint(0, min(self.pattern_size , other.pattern_size) - 1)
            if n > m:
                n,m = m,n
            self.pattern[n:m] = other.pattern[n:m]
        self.generate_genome()
        return self

    def one_cut_crossover(self,other : Agent) -> Agent:
        #print("scramble_mutation")
        n = random.randint(0, min(self.pattern_size , other.pattern_size) - 1)
        self.pattern[n:] = other.pattern[n:]
        self.generate_genome()
        return self
    
    def cycle_crossover(self,other : Agent) -> Agent:
        #print("cycle_crossover")
        RuntimeError("Not yet implemented")
        n = random.randint(0, len(self.genome)-1)
        m = random.randint(0, len(self.genome)-1)
        if n > m:
            n,m = m,n
        self.genome[n:m] = other.genome[n:m]
        return self

    def compute_fitness(self,fitness_function : AbstractProblem) -> None:
        super().compute_fitness(fitness_function)
    
    @property
    def fitness(self):
        return self._fitness

In [ ]:
def calculate_entropy(agent_list): #TODO rewrite
    # Calcola la probabilità di ciascun agente rispetto al totale
    total_agents = len(agent_list)
    probabilities = [agent.fitness / total_agents for agent in agent_list]

    # Calcola l'entropia utilizzando la formula di Shannon
    entropy = -sum(p * math.log2(p) if p > 0 else 0 for p in probabilities)

    return entropy

In [ ]:
def average_rule(agent_list : list):
    # Calcola la media delle regole di tutti gli agenti
    total_agents = len(agent_list)
    total_rules = len(agent_list[0].genome)
    average = [0 for _ in range(total_rules)]

    for agent in agent_list:
        for i in range(total_rules):
            average[i] += agent.genome[i]

    average = [x / total_agents for x in average]

    return average

In [ ]:
class AgentIslandsTraining():

    def __init__(self, pop_size : int,problem  : AbstractProblem,k : int = 50,islands_number : int = 10,lam : int = None,agent_type : Agent.AgentType = Agent.AgentType.BINARY) -> None:
        self.state = problem
        self._starting_mutation_rate = 0.008
        self.ration = 0.8
        self.mutation_rate = self._starting_mutation_rate
        self.old_fitness = None
        self.static_epochs = 0
        self.islands = [ [] for _ in range(islands_number)]
        self._island_population = pop_size // islands_number
        self._random_restart_probability = 0
        self.lam = lam if lam is not None else pop_size // islands_number
        print("Creating Islands")
        print(f"Islands : {self.islands}")

        match agent_type:
            case Agent.AgentType.BINARY:
                self._agent = AgentGA
            case Agent.AgentType.PATTERN:
                self._agent = PatternBasedAgent

        for island in self.islands:
            for _ in range(pop_size // islands_number):
                temp : Agent = self._agent().set_genome(genome_size=k).set_mutation_rate(random.uniform(0.001, 0.006))
                temp.compute_fitness(self.state)
                island.append(temp)
            print(f"Islands : {[len(x) for x in self.islands]}")

############################################################################################################
#   Parallel version of the algorithm
#   TODO : fix the parallel version
############################################################################################################

    def process_island(self,args):
        i, island, size, ration, state = args
        for _ in range(size):
            parent, other = random.choices(island, k=2)
            parent = deepcopy(parent)
            if random.random() > ration:
                parent.crossover(other, Agent.Crossover.MULTI_CUT)
            parent.mutation()
            parent.compute_fitness(state)
            island.append(parent)
        island.sort(key=lambda x: x.fitness, reverse=True)
        return island[:size]

    def parallel_generation(self):
        args_list = [(i, island, len(island), self.ration, self.state) for i, island in enumerate(self.islands)]

        with concurrent.futures.ThreadPoolExecutor() as executor:
            new_islands = list(executor.map(self.process_island, args_list))

        self.islands = new_islands

    def generation(self) -> None:
        for i , island in enumerate(self.islands):      
            #print(f"Island : {i} with {len(island)} pop")
            size = len(island)
            for _ in range(self.lam):
                #parent , other = random.choices(island,weights=[x.fitness for x in island],k=2)
                parent , other = random.choices(island,k=2)
                #print(f"parent : {parent} , other : {other}")
                parent = deepcopy(parent)
                if random.random() > self.ration :
                    parent.crossover(other,Agent.Crossover.MULTI_CUT)
                #parent.mutation(self.mutation_rate / 2)
                parent.mutation()
                parent.compute_fitness(self.state)
                island.append(parent)    
            island.sort(key=lambda x : x.fitness , reverse=True)
            self.islands[i] = island[:size]
            if self._random_restart_probability > random.random():
                self.islands[i][-1] = self._agent().set_genome(genome_size=len(self.islands[i][-1].genome)).set_mutation_rate(random.uniform(0.001, 0.006))
                self.islands[i][-1].compute_fitness(self.state)
                self._random_restart_probability = 0.9 * self._random_restart_probability
            #print(f"Islands : {[len(x) for x in self.islands]} at iteration {i}")
            #print(f"Island : {i} with {len(island)} pop")
    
    def generation_tournament(self) -> None:
        for i , island in enumerate(self.islands):      
            current_member = 0
            mating_pool = []
            while current_member < self.lam:
                tournament = random.choices(island,k=self.tournament_size)
                tournament.sort(key=lambda x : x.fitness , reverse=True)
                mating_pool.append(tournament[0])
                current_member += 1
            temp_population = []
            for parent in mating_pool:
                for _ in range(self.lam):
                    temp : Agent = deepcopy(parent).mutation(self.ration)
                    temp.compute_fitness(self.state)
                    temp_population.append(temp)  
            temp_population.sort(key=lambda x : x.fitness , reverse=True)
            self.islands[i][-self.lam:] = temp_population[:self.lam]

        #define what to do if num_islands is odd
    def migration(self , N : int = 1):
        if len(self.islands) == 1:
            return
        for i in range(len(self.islands)-1) :
            for _ in range(N):
                pos_1 = random.randint(0,len(self.islands[i]) - 1)
                pos_2 = random.randint(0,len(self.islands[(i+1)%len(self.islands)]) - 1)
                self.islands[i][pos_1] , self.islands[(i+1)%len(self.islands)][pos_2] =  self.islands[(i+1)%len(self.islands)][pos_2] , self.islands[i][pos_1]
               

    def train(self,generations = 10):
        best_fitness = None
        print("Starting Training")
        for gen in range(generations):
            #print(f"Islands : {[len(x) for x in self.islands]} at gen {gen}")
            self.generation()
            #print(f"Islands : {[len(x) for x in self.islands]} at gen {gen}")
            if (gen + 1 ) % 10 == 0:
                best_fitness = max([agent for island in self.islands for agent in island], key=lambda agent: agent.fitness)
                self.migration(self._island_population // 10)
                if self.old_fitness == best_fitness:
                    self.static_epochs += 1
                    self._random_restart_probability = 0.9 * self._random_restart_probability + 0.1
                else:
                    self.static_epochs = 0
                    self.old_fitness = best_fitness
                    self._random_restart_probability = 0
                if self.static_epochs > 2:
                    self.mutation_rate = min(self._starting_mutation_rate,1.05 * self.mutation_rate)
                    #self.ration = max(0.6,1.1 * self.ration)
                else:
                    self.mutation_rate = max(self._starting_mutation_rate/10,0.95 * self.mutation_rate)
                    self.ration = max(0.4,0.99 * self.ration)
                print(f"Best fitness : {best_fitness.fitness} at gen {gen+1} with criterion : {self.ration} , mutation : {best_fitness.mutation_rate} , static_epochs : {self.static_epochs}")
                temp = average_rule([agent for island in self.islands for agent in island])
                print(f"Average sum : {sum(temp) / 1000} , rule weight : {temp}")
                #print(f"Best fitness : {best_fitness.fitness} at gen {gen} with total population : {([calculate_entropy(x) for x in self.islands])} , criterion : {self.ration}")
            #self.islands.sort(key=lambda x : x[0].fitness , reverse=True)
            if best_fitness is not None and best_fitness.fitness == 1.0:
                break
        #print([island for island in self.islands])
        return best_fitness

In [ ]:
fitness = lab3_lib.make_problem(1)
trainer = AgentIslandsTraining(400,fitness,k=1000,islands_number=5,agent_type=Agent.AgentType.PATTERN)
best_agent = trainer.train(5000)
print(best_agent)
fitness.calls

#### A dumb approach to the problem
Trying to optimize the problem by trying to maximize the fitness bit-by-bit, converge almost instantly for n = 1, but can't find anything with 2,5,10 problem size

In [ ]:
def dummy_solution(fitness):
    k = 1000
    agent_slighly_smarter = [random.uniform(0,1) < 0.5 for _ in range(k)]
    agent_fitness = fitness(agent_slighly_smarter)
    while agent_fitness < 1.0:
        for i in range(k):
            temp = deepcopy(agent_slighly_smarter)
            temp[i] = not temp[i]
            temp_fitness = fitness(temp)
            if temp_fitness > agent_fitness:
                agent_slighly_smarter = temp
                agent_fitness = temp_fitness
        print(agent_fitness)
    #print(fitness(agent_slighly_smarter))
    #print(fitness.calls)
    return agent_slighly_smarter
fitness = lab3_lib.make_problem(1)
print(fitness(dummy_solution(fitness)))

In [ ]:
def hill_climbing(k : int,fitness : AbstractProblem,lam):
    agent = PatternBasedAgent().set_genome(genome_size=k,genome=[random.uniform(0,1) < 0.5 for _ in range(k)])
    agent.compute_fitness(fitness)
    gen = 0
    gene_modifier = 2
    mutation_rate = gene_modifier / k
    while agent.fitness < 1.0:
        offspring = []
        for _ in range(lam):
            agent_temp = deepcopy(agent)
            agent_temp.mutation(mutation_rate)
            agent_temp.compute_fitness(fitness)
            offspring.append(agent_temp)
        offspring.append(agent)
        offspring.sort(key=lambda x : x.fitness , reverse=True)
        agent = offspring[0]
        gen += 1
        if gen % 10 == 0:
            print(f"gen {gen} : Best fitness : {agent.fitness}")

In [ ]:
fitness = lab3_lib.make_problem(1)
hill_climbing(1000,fitness,40)
fitness.calls